<a href="https://colab.research.google.com/github/faisal-ai/Sarcastic-News-Headlines-Detection-Using-Support-Vector-Classifier/blob/main/Sarcastic_News_Headlines_Detection_using_SVC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import sklearn
import numpy as np
import pandas as pd

In [5]:
# training data
train = pd.read_csv("/content/traind.csv")

# test data
test = pd.read_csv("/content/testd.csv")

# Exploratory Data Analysis

In [ ]:
train.head()

,is_sarcastic,headline
0,1,thirtysomething scientists unveil doomsday clo...
1,0,dem rep. totally nails why congress is falling...
2,0,eat your veggies: 9 deliciously different recipes
3,1,inclement weather prevents liar from getting t...
4,1,mother comes pretty close to using word 'strea...


In [6]:
test.head()

,is_sarcastic,headline
0,1,bush proud u.s. economic woes can still depres...
1,1,chris pine depressed by realization he could p...
2,1,it's shark week!
3,1,police found golden state killer by tracing ow...
4,0,virgin galactic is helping develop a new super...


In [7]:
#num of non-sarcastic headlines
sum(train["is_sarcastic"] == 0)

11661

In [8]:
#num of sarcastic headlines
sum(train["is_sarcastic"] == 1)

10636

In [9]:
# checking missing values
train.isnull().sum()

is_sarcastic    0
headline        0
dtype: int64

# Data cleaning

In [10]:

!pip install tweet-preprocessor

In [11]:
# remove special characters using the regular expression library
import re


REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\|)|(\()|(\))|(\[)|(\])|(\%)|(\$)|(\>)|(\<)|(\{)|(\})")
REPLACE_WITH_SPACE = re.compile("(<br\s/><br\s/?)|(-)|(/)|(:).")

In [12]:
import preprocessor as p

# custum function to clean the dataset (combining tweet_preprocessor and reguar expression)
def clean_tweets(df):
  tempArr = []
  for line in df:
    # send to tweet_processor
    tmpL = p.clean(line)
    # remove puctuation
    tmpL = REPLACE_NO_SPACE.sub("", tmpL.lower()) # convert all tweets to lower cases
    tmpL = REPLACE_WITH_SPACE.sub(" ", tmpL)
    tempArr.append(tmpL)
  return tempArr

In [13]:
# clean training data
train_headline = clean_tweets(train["headline"])
train_headline = pd.DataFrame(train_headline)

In [14]:
# append cleaned headlines to the training data
train["clean_headline"] = train_headline

# compare the cleaned and uncleaned headlines
train.head(10)

,is_sarcastic,headline,clean_headline
0,1,thirtysomething scientists unveil doomsday clo...,thirtysomething scientists unveil doomsday clo...
1,0,dem rep. totally nails why congress is falling...,dem rep totally nails why congress is falling ...
2,0,eat your veggies: 9 deliciously different recipes,eat your veggies deliciously different recipes
3,1,inclement weather prevents liar from getting t...,inclement weather prevents liar from getting t...
4,1,mother comes pretty close to using word 'strea...,mother comes pretty close to using word stream...
5,0,my white inheritance,my white inheritance
6,0,5 ways to file your taxes with less stress,ways to file your taxes with less stress
7,1,richard branson's global-warming donation near...,richard bransons global warming donation nearl...
8,1,shadow government getting too large to meet in...,shadow government getting too large to meet in...
9,0,lots of parents know this scenario,lots of parents know this scenario


In [15]:
# cleaning the test data and appending the cleaned headlines to the test data
test_headline = clean_tweets(test["headline"])
test_headline = pd.DataFrame(test_headline)

test["clean_headline"] = test_headline

# compare the cleaned and uncleaned headlines
test.head()

,is_sarcastic,headline,clean_headline
0,1,bush proud u.s. economic woes can still depres...,bush proud us economic woes can still depress ...
1,1,chris pine depressed by realization he could p...,chris pine depressed by realization he could p...
2,1,it's shark week!,its shark week
3,1,police found golden state killer by tracing ow...,police found golden state killer by tracing ow...
4,0,virgin galactic is helping develop a new super...,virgin galactic is helping develop a new super...


# Test and Train split


In [17]:
from sklearn.model_selection import train_test_split

y = train.is_sarcastic.values

# using 70% for the training and 30% for the testing
x_train, x_test, y_train, y_test = train_test_split(train.clean_headline.values, y, 
                                                   
                                                    random_state=1, 
                                                    test_size=0.3, shuffle=True)

# Vectorize tweets using CountVectorizer

CountVectorizer Example

In [18]:
from sklearn.feature_extraction.text import CountVectorizer

# vectorize tweets for model building
vectorizer = CountVectorizer(binary=True, stop_words='english')

# learn a vocabulary dictionary of all tokens in the raw documents
vectorizer.fit(list(x_train) + list(x_test))

# transform documents to document-term matrix
x_train_vec = vectorizer.transform(x_train)
x_test_vec = vectorizer.transform(x_test)

# Model building

Classifying using Support Vetor Classifier (SVC)

In [19]:
from sklearn import svm
# classify using support vector classifier
svm = svm.SVC(kernel = 'linear', probability=True)

# fit the SVC model based on the given training data
prob = svm.fit(x_train_vec, y_train).predict_proba(x_test_vec)

# perform classification and prediction on samples in x_test
y_pred_svm = svm.predict(x_test_vec)

# Accuracy score of the model


In [20]:
from sklearn.metrics import accuracy_score
print("Accuracy score for SVC is: ", accuracy_score(y_test, y_pred_svm) * 100, '%')

Accuracy score for SVC is:  77.27952167414051 %
